In [1]:
import numpy as np
import pandas as pd
import gymnasium as gym
import os

In [2]:
print(os.getcwd())
os.chdir('../../')
print(os.getcwd())

/home/vanishhhed/Desktop/codics/PythonCodics/ML/RL/music_rec_system/code/environment
/home/vanishhhed/Desktop/codics/PythonCodics/ML/RL/music_rec_system


In [3]:
from config import DATA_CLEANED_PATH, DATA_PREP_PATH, DATA_RAW_PATH

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
# Настройки
OUTPUT_PATH = 'data/prep/sessions_tied.parquet'
TIED_WINDOW = 2  # Окно событий для проверки (±2)

def clean_tied_events(row):
    events = np.array(row['event_type'])
    item_ids = np.array(row['item_ids'])
    played_ratios = np.array(row['played_ratio_pct']) if row['played_ratio_pct'] is not None else np.full(len(events), None)
    
    # Маска для сохранения: listen или tied non-listen (с совпадением item_id)
    keep_mask = np.zeros(len(events), dtype=bool)
    for i in range(len(events)):
        if events[i] == 'listen':
            keep_mask[i] = True
            continue
        # Проверяем окно на listen с ТЕМ ЖЕ item_id
        current_id = item_ids[i]
        window_start = max(0, i - TIED_WINDOW)
        window_end = min(len(events), i + TIED_WINDOW + 1)
        has_tied_listen = any(
            events[j] == 'listen' and item_ids[j] == current_id
            for j in range(window_start, window_end)
        )
        if has_tied_listen:
            keep_mask[i] = True
    
    # Если маска пустая — skip сессию
    if not np.any(keep_mask):
        return None
    
    # Фильтруем
    filtered = {
        'uid': row['uid'],
        'session_idx': row['session_idx'],
        'session_length': int(np.sum(keep_mask)),
        'item_ids': item_ids[keep_mask].tolist(),
        'played_ratio_pct': played_ratios[keep_mask].tolist() if played_ratios is not None else None,
        'event_type': events[keep_mask].tolist(),
    }
    return filtered

# Основной цикл
print("Загружаем сессии...")
sessions_df = pd.read_parquet(DATA_CLEANED_PATH)

cleaned_sessions = []
for _, row in tqdm(sessions_df.iterrows(), total=len(sessions_df), desc="Очистка сессий"):
    cleaned = clean_tied_events(row)
    if cleaned:
        cleaned_sessions.append(cleaned)

cleaned_df = pd.DataFrame(cleaned_sessions)
cleaned_df.to_parquet(OUTPUT_PATH, index=False, compression='snappy')

print(f"Готово! Новый файл: {OUTPUT_PATH}")
print(f"Сессий до: {len(sessions_df)}, после: {len(cleaned_df)}")
print(f"Средняя длина: {cleaned_df['session_length'].mean():.2f}")

Загружаем сессии...


In [ ]:
import pandas as pd
import numpy as np

# Настройки
INPUT_PATH = OUTPUT_PATH
OUTPUT_PATH = 'data/prep/sessions_clipped.parquet'

def clip_played_ratios(row):
    played_ratios = row['played_ratio_pct']
    if played_ratios is None:
        return row  # Если вся колонка None — не трогаем
    
    # Конвертируем в numpy array, clip только not None
    ratios_array = np.array(played_ratios)
    # Маска для not None
    mask = ~pd.isnull(ratios_array)
    if np.any(mask):
        ratios_array[mask] = np.clip(ratios_array[mask], a_min=0, a_max=100)
    
    # Обновляем row
    updated_row = row.copy()
    updated_row['played_ratio_pct'] = ratios_array.tolist()
    return updated_row

# Основной цикл
print("Загружаем сессии...")
sessions_df = pd.read_parquet(INPUT_PATH)

updated_sessions = []
for _, row in tqdm(sessions_df.iterrows(), total=len(sessions_df), desc="Нормализация значений"):
    updated = clip_played_ratios(row.to_dict())
    if updated is not None:  # На случай, если skip (но здесь всегда сохраняем)
        updated_sessions.append(updated)

updated_df = pd.DataFrame(updated_sessions)
updated_df.to_parquet(OUTPUT_PATH, index=False, compression='snappy')

print(f"Готово! Новый файл: {OUTPUT_PATH}")
print(f"Сессий: {len(updated_df)}")
print(f"Пример первой строки: {updated_df.iloc[0]}")

Загружаем сессии...
Готово! Новый файл: yambda_sessions_clipped.parquet
Сессий: 14
Пример первой строки: uid                                                        1
session_idx                                              111
item_ids                                [98, 99, 67, 69, 54]
played_ratio_pct            [100.0, 100.0, 80.0, 40.0, 20.0]
event_type          [listen, listen, listen, listen, listen]
session_length                                             5
Name: 0, dtype: object
